In [1]:
import tensorflow as tf
import helper_functions as hf
print(tf.__version__)

2.9.1


In [2]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=r'D:\1jupyter\Datasets\101_food_classes_10_percent\train',
                                                                 image_size=(224,224),
                                                                 batch_size=32,
                                                                 label_mode='categorical')

test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=r'D:\1jupyter\Datasets\101_food_classes_10_percent\test',
                                                                image_size=(224,224),
                                                                batch_size=32,
                                                                label_mode='categorical')

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [3]:
checkpoint_path = 'checkpoints/all_classes_checkpoint.ckpt'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_weights_only=True,
                                                         save_best_only=False,
                                                         save_freq='epoch',
                                                         verbose=1)

In [4]:
data_augmentaion = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomHeight(0.2),
    tf.keras.layers.experimental.preprocessing.RandomWidth(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
], name='augmentation_layer')

In [5]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable=True
for layer in base_model.layers[:-10]:
    layer.trainable=False

inputs = tf.keras.layers.Input(shape=(224,224,3), name='input_layer')
x = data_augmentaion(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name='pooling_layer')(x)
outputs = tf.keras.layers.Dense(units=101, activation='softmax', name='output_layer')(x)

model1 = tf.keras.Model(inputs,outputs)

In [6]:
for layer in base_model.layers[:-10]:
    layer.trainable=False

In [7]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 rescaling False
2 normalization False
3 tf.math.truediv False
4 stem_conv_pad False
5 stem_conv False
6 stem_bn False
7 stem_activation False
8 block1a_dwconv False
9 block1a_bn False
10 block1a_activation False
11 block1a_se_squeeze False
12 block1a_se_reshape False
13 block1a_se_reduce False
14 block1a_se_expand False
15 block1a_se_excite False
16 block1a_project_conv False
17 block1a_project_bn False
18 block2a_expand_conv False
19 block2a_expand_bn False
20 block2a_expand_activation False
21 block2a_dwconv_pad False
22 block2a_dwconv False
23 block2a_bn False
24 block2a_activation False
25 block2a_se_squeeze False
26 block2a_se_reshape False
27 block2a_se_reduce False
28 block2a_se_expand False
29 block2a_se_excite False
30 block2a_project_conv False
31 block2a_project_bn False
32 block2b_expand_conv False
33 block2b_expand_bn False
34 block2b_expand_activation False
35 block2b_dwconv False
36 block2b_bn False
37 block2b_activation False
38 block2b_se_squeeze Fals

In [8]:
model1.compile(loss='categorical_crossentropy',
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
               metrics=['accuracy'])

In [9]:
history1 = model1.fit(train_data, epochs=5, 
                      steps_per_epoch=len(train_data),
                      validation_data=test_data,
                      validation_steps=int(0.15*(len(test_data))),
                      callbacks=[hf.create_tensorboard_callback(dir_name='logs', experiment_name='all_data_10_percent'),
                                 checkpoint_callback])

Saving TensorBoard log files to: logs/all_data_10_percent/20220813-214306
Epoch 1/5
237/237 [==============================] - ETA: 0s - loss: 4.0184 - accuracy: 0.1331
Epoch 1: saving model to checkpoints\all_classes_checkpoint.ckpt
237/237 [==============================] - 151s 566ms/step - loss: 4.0184 - accuracy: 0.1331 - val_loss: 2.8034 - val_accuracy: 0.3438
Epoch 2/5
237/237 [==============================] - ETA: 0s - loss: 2.5440 - accuracy: 0.3810
Epoch 2: saving model to checkpoints\all_classes_checkpoint.ckpt
237/237 [==============================] - 98s 412ms/step - loss: 2.5440 - accuracy: 0.3810 - val_loss: 2.0591 - val_accuracy: 0.4725
Epoch 3/5
237/237 [==============================] - ETA: 0s - loss: 2.0476 - accuracy: 0.4851
Epoch 3: saving model to checkpoints\all_classes_checkpoint.ckpt
237/237 [==============================] - 98s 411ms/step - loss: 2.0476 - accuracy: 0.4851 - val_loss: 1.8300 - val_accuracy: 0.5164
Epoch 4/5
237/237 [========================

In [10]:
model1.evaluate(test_data)

790/790 [==============================] - 92s 116ms/step - loss: 1.6910 - accuracy: 0.5502


[1.690964698791504, 0.5502178072929382]

In [11]:
model1.save('all_food_10_percent')

INFO:tensorflow:Assets written to: all_food_10_percent\assets


INFO:tensorflow:Assets written to: all_food_10_percent\assets


In [12]:
loaded_model = tf.keras.models.load_model('all_food_10_percent')

In [14]:
loaded_model.evaluate(test_data)

790/790 [==============================] - 89s 110ms/step - loss: 1.6910 - accuracy: 0.5502


[1.690964937210083, 0.5502178072929382]

In [15]:
pred_probs = model1.predict(test_data,verbose=1)

790/790 [==============================] - 93s 115ms/step


In [16]:
pred_probs.shape

(25250, 101)

In [17]:
import numpy as np
np.argmin(pred_probs[0])

42

In [18]:
test_data.class_names[74]

'peking_duck'

In [19]:
pred_probs[0][74]

2.2825996e-05

In [20]:
pred_classes = pred_probs.argmax(axis=1)

In [21]:
pred_classes[:10]

array([11, 33, 48, 73, 25, 79, 98, 37, 33, 78], dtype=int64)

In [22]:
y_labels=[]
for image, labels in test_data.unbatch():
    y_labels.append(labels.numpy().argmax())
y_labels[:10]

[25, 75, 58, 35, 66, 81, 62, 71, 81, 81]

In [23]:
from sklearn.metrics import accuracy_sore
accuracy_score(y_labels, pres_classes)

ImportError: cannot import name 'accuracy_sore' from 'sklearn.metrics' (D:\anaconda\envs\tensorflow\lib\site-packages\sklearn\metrics\__init__.py)

In [ ]:
class_names = test_data.class_names